# Importing Necessary Libraries

In [4]:
!pip install -q "requests==2.32.4" \
               "opentelemetry-api==1.37.0" \
               "opentelemetry-sdk==1.37.0" \
               "opentelemetry-exporter-otlp-proto-common==1.37.0" \
               "opentelemetry-proto==1.37.0"


In [ ]:
!pip install -q --upgrade \
  langchain \
  langchain-community \
  chromadb \
  pypdf \
  langchain-text-splitters \
  sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 40.2 MB/s eta 0:00:

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading and Splitting


*   Load the PDF documents from a directory
*   Split the documents into smaller chunks using the RecursiveCharacterTextSplitter



In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

# pdf_path = "/content/drive/MyDrive/GenAI_Papers.pdf"
# adjust if in a subfolder

pdf_folder = "/content/drive/MyDrive/GenAI_Papers"  # change if yours is in a subfolder

documents = []
for file_name in os.listdir(pdf_folder):
    if file_name.endswith(".pdf"):
        pdf_file_path = os.path.join(pdf_folder, file_name)
        loader = PyPDFLoader(pdf_file_path)
        documents.extend(loader.load())


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
texts = text_splitter.split_documents(documents)

In [8]:
type(texts)

list

In [ ]:
# Using open-source HuggingFace embeddings - no API key needed!
print("Using open-source HuggingFace embeddings (no API key required).")

OpenAI API key loaded successfully from Colab secrets.


# Loading the HuggingFace Embeddings (Open Source)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Using open-source HuggingFace embeddings - no API key needed!
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Creating and Loading the Database


*   Create a database to store the embedded text
*   Load the database to bring it back into memory from the disk



In [ ]:
from langchain_community.vectorstores import Chroma

#Creating a database

persist_directory = 'db'
embedding = hf_embeddings
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()
vectordb = None
#Loading the database
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}